In [1]:
# For easier dev of local modules:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import json

# External Dependencies:
import boto3
import sagemaker
from sagemaker.pytorch.estimator import PyTorch as PyTorchEstimator


In [2]:
BUCKET_NAME = "2020-05-gym-bert"
%store BUCKET_NAME

SQUAD_V2 = False  # Whether to use V2 (including unanswerable questions)
%store SQUAD_V2

Stored 'BUCKET_NAME' (str)
Stored 'SQUAD_V2' (bool)


In [3]:
role = sagemaker.get_execution_role()
botosess = boto3.session.Session()
region = botosess.region_name
s3 = botosess.resource("s3")
bucket = s3.Bucket(BUCKET_NAME)
smclient = botosess.client("sagemaker")

In [4]:
!mkdir -p models

In [5]:
bucket.Object(
    "bert-calssification-distributed-2020-05-05-15-58-03-622/output/output.tar.gz"
).download_file(
    "models/bert-cls.tar.gz"
)
bucket.Object(
    "distilbert-calssification-distributed-2020-05-05-16-24-55-728/output/output.tar.gz"
).download_file(
    "models/distilbert-cls.tar.gz"
)

In [ ]:
!mkdir -p models/bert-cls
!tar -C models/bert-cls -zxvf models/bert-cls.tar.gz
!mkdir -p models/distilbert-cls
!tar -C models/distilbert-cls -zxvf models/distilbert-cls.tar.gz

In [ ]:
# Install AWS SAM
!./install-sam.sh

In [ ]:
# Create an empty virtual env, install PyTorch, then extract it into lambda/packages
!./configure-venv.sh

# Might need to do this if re-running:
# !conda env remove -n lambda_bert -y

In [3]:
# Deploy the Lambda + API Gateway:
STAGING_BUCKET = "2020-05-gym-bert-sam-staging"
STACK_NAME = "testa"
!./deploy.sh {STAGING_BUCKET} {STACK_NAME}